In [1]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import boto3
import pandas as pd
from dotenv import load_dotenv
import os
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
import warnings
import numpy as np
from mlflow.models.signature import infer_signature
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
load_dotenv()

access_key = os.getenv("AWS_ACCESS_KEY_ID")
secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
s3_url = os.getenv("MLFLOW_S3_ENDPOINT_URL")
tracker_url = os.getenv("MLFLOW_URL")
os.environ["MLFLOW_S3_ENDPOINT_URL"] = os.getenv("MLFLOW_S3_ENDPOINT_URL")
os.environ["MLFLOW_TRACKING_USERNAME"] = os.getenv("MLFLOW_ADMIN_USERNAME")
os.environ["MLFLOW_TRACKING_PASSWORD"] = os.getenv("MLFLOW_ADMIN_PASSWORD")
os.environ["MLFLOW_S3_IGNORE_TLS"] = "true"
os.environ["MLFLOW_ENABLE_SYSTEM_METRICS_LOGGING"] = "true"
bucket_name = "dataset"
object_name = "water_potability.csv"

In [3]:
s3 = boto3.client(
    's3',
    endpoint_url=s3_url,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    verify=False
)

In [4]:
try:
    # Attempt to list buckets
    response = s3.list_buckets()
    
    # If successful, print bucket names
    print("Connected successfully! Buckets available:")
    for bucket in response['Buckets']:
        print(f"- {bucket['Name']}")
except Exception as e:
    # Print any connection errors
    print("Connection error:", e)


Connected successfully! Buckets available:
- dataset
- mlflow


In [19]:
try:
    response = s3.get_object(Bucket=bucket_name, Key=object_name)
    print(response)
    dataset_content = response.get('Body')
    print(dataset_content)
    # Load the dataset into a DataFrame
    data = pd.read_csv(dataset_content)
    print("Dataset loaded successfully:")
    print(data.head())
except Exception as e:
    print("Error fetching dataset from MinIO:", e)


{'ResponseMetadata': {'RequestId': '1807C2ACF556570D', 'HostId': 'b675f5c02385af78c69266a96b22038bea3ecd7423814977a7267fd68ac2bbeb', 'HTTPStatusCode': 200, 'HTTPHeaders': {'accept-ranges': 'bytes', 'content-length': '525187', 'content-type': 'text/csv', 'etag': '"a5eeae83d792e40da6c2f9e1eab945fc"', 'last-modified': 'Tue, 12 Nov 2024 07:15:22 GMT', 'server': 'MinIO', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'vary': 'Origin, Accept-Encoding', 'x-amz-id-2': 'b675f5c02385af78c69266a96b22038bea3ecd7423814977a7267fd68ac2bbeb', 'x-amz-request-id': '1807C2ACF556570D', 'x-content-type-options': 'nosniff', 'x-ratelimit-limit': '851', 'x-ratelimit-remaining': '851', 'x-xss-protection': '1; mode=block', 'x-amz-version-id': 'ae37a965-df07-433c-85ff-3d7ac1b462ef', 'date': 'Thu, 14 Nov 2024 06:43:49 GMT'}, 'RetryAttempts': 0}, 'AcceptRanges': 'bytes', 'LastModified': datetime.datetime(2024, 11, 12, 7, 15, 22, tzinfo=tzutc()), 'ContentLength': 525187, 'ETag': '"a5eeae83d792e

In [20]:
df = data[['ph','Solids','Potability']]

In [21]:
df['Solids'] = (df['Solids'] - df['Solids'].min()) / (df['Solids'].max() - df['Solids'].min()) * 1000

/var/folders/4f/xq7119ld627_2wyd6dkmwl0m0000gn/T/ipykernel_65298/1407353664.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Solids'] = (df['Solids'] - df['Solids'].min()) / (df['Solids'].max() - df['Solids'].min()) * 1000


In [22]:
df.describe()

,ph,Solids,Potability
count,2785.000000,3276.000000,3276.000000
mean,7.080795,356.172785,0.390110
std,1.594320,143.968318,0.487849
min,0.000000,0.000000,0.000000
25%,6.093092,251.956849,0.000000
50%,7.036752,338.337853,0.000000
75%,8.062066,443.498295,1.000000
max,14.000000,1000.000000,1.000000


In [9]:
mlflow.set_tracking_uri(tracker_url)
experiment_name = "water_potability"
mlflow.set_experiment(experiment_name)

data = df.drop('Potability', axis=1)
target = df['Potability']
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

In [10]:
from mlflow.models.signature import infer_signature
import mlflow.xgboost
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

# Model parameters
n_estimators = 100
max_depth = 5
random_state = 42

with mlflow.start_run():
    # Log parameters
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("random_state", random_state)

    # Initialize and train the model
    model = XGBClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state)
    model.fit(X_train, y_train)

    # Evaluate the model
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    mlflow.log_metric("accuracy", accuracy)

    # Define input example and signature
    input_example = X_test.iloc[:1].fillna(X_test.mean())
    signature = infer_signature(X_test, model.predict(X_test))

    # Log the model with MLflow
    mlflow.xgboost.log_model(
        model,
        artifact_path="model",
        input_example=input_example,
        signature=signature
    )

    # Print run information
    run_id = mlflow.active_run().info.run_id
    print(f"Run ID: {run_id}")
    print(f"Model accuracy: {accuracy}")


2024/11/14 11:24:48 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: Failed to initialize NVML, skip logging GPU metrics: NVML Shared Library Not Found.
2024/11/14 11:24:48 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
/Users/fauzanghaza/Applications/miniconda3/envs/MLOps/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [11:24:49] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1724807611129/work/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
/Users/fauzanghaza/Applications/miniconda3/envs/MLOps/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import

Run ID: 0a3b45725937468394fd8df9d45a9ed3
Model accuracy: 0.573170731707317


2024/11/14 11:24:54 INFO mlflow.tracking._tracking_service.client: 🏃 View run mercurial-roo-165 at: http://10.34.4.198:5000/#/experiments/1/runs/0a3b45725937468394fd8df9d45a9ed3.
2024/11/14 11:24:54 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://10.34.4.198:5000/#/experiments/1.
2024/11/14 11:24:54 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/11/14 11:24:55 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


In [13]:
import mlflow.pyfunc

model_uri = f"runs:/{run_id}/model"
model = mlflow.pyfunc.load_model(model_uri)

# Check if metadata and signature are available
if model.metadata:
    print("Model signature:", model.metadata.get_input_schema())
else:
    print("Model metadata is missing.")

Model signature: ['ph': double (optional), 'Solids': double (required)]


In [12]:
from mlflow.models.signature import infer_signature
import mlflow.xgboost
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

# Model parameters
n_estimators = 100
max_depth = 5
random_state = 42

with mlflow.start_run():
    # Log parameters
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("random_state", random_state)

    # Initialize and train the model
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state)
    model.fit(X_train, y_train)

    # Evaluate the model
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    mlflow.log_metric("accuracy", accuracy)

    # Define input example and signature
    input_example = X_test.iloc[:1].fillna(X_test.mean())
    signature = infer_signature(X_test, model.predict(X_test))

    # Log the model with MLflow
    mlflow.sklearn.log_model(
        model,
        artifact_path="model",
        input_example=input_example,
        signature=signature
    )

    # Print run information
    run_id = mlflow.active_run().info.run_id
    print(f"Run ID: {run_id}")
    print(f"Model accuracy: {accuracy}")


2024/11/14 11:24:56 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: Failed to initialize NVML, skip logging GPU metrics: NVML Shared Library Not Found.
2024/11/14 11:24:56 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


Run ID: 111f57b819d9456c8c0b8e4a2a306c2d
Model accuracy: 0.6234756097560976


2024/11/14 11:25:01 INFO mlflow.tracking._tracking_service.client: 🏃 View run chill-penguin-562 at: http://10.34.4.198:5000/#/experiments/1/runs/111f57b819d9456c8c0b8e4a2a306c2d.
2024/11/14 11:25:01 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://10.34.4.198:5000/#/experiments/1.
2024/11/14 11:25:01 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/11/14 11:25:01 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!
